# Prerequisites

In [2]:
import numpy as np
import pandas as pd
from tm import StockDataProvider
from tm.trading_rules import SimpleMovingAverage, ExponentialMovingAverage, STO, MACD, RSI, ROC
from tm.optimizers import GeneticOptimizer, StrategyPerformanceEvaluator, map_chromosome_to_trading_rule_parameters, filter_for_active_rules, calculate_absolute_buy_and_hold_returns
from tm.backtesting import GeometricBrownianMotion, MonteCarloCrossValidation
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

Define the trading rule basis for the strategy.

In [3]:
trading_rules = [SimpleMovingAverage, ExponentialMovingAverage, STO, MACD, RSI, ROC]

Define a function which finds the best strategy by running the genetic algorithm and applying backtesting.

In [4]:
# Returns best strategy as bitvector
def find_best_strategy(ticker_name, start, end, ngen=5):
    data = StockDataProvider(ticker_name, start=start, end=end)
    optimizer = GeneticOptimizer(data, trading_rules, random_seed=42)
    print('Running genetic algorithm ...')
    hof = optimizer.run(pop_size=1000, ngen=ngen, hof_size=100)
    print('Net profit of buy and hold:', calculate_absolute_buy_and_hold_returns(data))
    gbm = GeometricBrownianMotion(data)
    crossvalidator = MonteCarloCrossValidation(hof, gbm, trading_rules)
    print('Running backtesting ...')
    best_individual = crossvalidator.run(num_iterations=100, time_steps=85)
    del optimizer
    return best_individual

Define a function which prints the results for the testing period.

In [ ]:
def print_test_results(individual, ticker_name, start, end):
    real_data = StockDataProvider(ticker_name, start=start, end=end)
    rule_instances = list(map(lambda Rule, params: Rule(real_data, *params), trading_rules, map_chromosome_to_trading_rule_parameters(individual, trading_rules)))
    active_rule_instances = filter_for_active_rules(individual, rule_instances)
    evaluator = StrategyPerformanceEvaluator(active_rule_instances)
    net_profit = evaluator.calculate_net_profit()
    net_profit_bh = calculate_absolute_buy_and_hold_returns(real_data)
    percentage_profit = net_profit / real_data.history['Close'].iloc[0] * 100
    percentage_profit_bh = net_profit_bh / real_data.history['Close'].iloc[0] * 100
    print('Active rules:', list(map(lambda rule: rule.__class__.__name__, active_rule_instances)))
    print('Active rules\' parameters:', list(map(lambda param: param[1], filter(lambda param: trading_rules[param[0]] in map(lambda rule: rule.__class__, active_rule_instances), enumerate(map_chromosome_to_trading_rule_parameters(individual, trading_rules))))))
    print('Net profit of strategy: {:f}'.format(net_profit))
    print('Net profit of buy and hold: {:f}'.format(net_profit_bh))
    print('Percentage profit of strategy: {:f}%'.format(percentage_profit))
    print('Percentage profit of buy and hold: {:f}%'.format(percentage_profit_bh))
    print('Number of buy signals:', len(evaluator.buy_signals[evaluator.buy_signals == True]))
    print('Number of sell signals:', len(evaluator.sell_signals[evaluator.sell_signals == True]))
    return evaluator, percentage_profit, percentage_profit_bh

Define a function which can plot a strategy.

In [ ]:
def plot_strategy(evaluator, ticker_name, start, end):
    data = StockDataProvider(ticker_name, start=start, end=end)
    
    fig, ax = plt.subplots(figsize=(16, 8))
    
    ax.plot(data.history['Close'], label='Stock prices')

    if len(evaluator.buy_signals[evaluator.buy_signals == True]) > 0:
        buy_points = pd.Series(data=map(lambda index, price: price if evaluator.buy_signals.loc[index] == True else np.nan, data.history['Close'].index, data.history['Close']),
                               index=data.history['Close'].index)
        ax.scatter(x=buy_points.index, y=buy_points, color='green', marker='^', label='Buy signal')
    
    if len(evaluator.sell_signals[evaluator.sell_signals == True]) > 0:
        sell_points = pd.Series(data=map(lambda index, price: price if evaluator.sell_signals.loc[index] == True else np.nan, data.history['Close'].index, data.history['Close']),
                               index=data.history['Close'].index)
        ax.scatter(x=sell_points.index, y=sell_points, color='red', marker='v', label='Sell signal')

    ax.set_xlabel('Date')
    ax.set_ylabel('Stock price')
    ax.legend()
    fig.tight_layout()

# Experiments

In [ ]:
results = {'EXS1.DE': [],
           'ADS.DE': [],
           'ALV.DE': [],
           'BAS.DE': [],
           'DBK.DE': [],
           'DTE.DE': [],
           'SAP.DE': [],
           'DJI': [],
           'CSCO': [],
           'DD': [],
           'JPM': [],
           'MSFT': [],
           'NKE': [],
           'UNH': []}

## DAX

In [6]:
%%time
best_individual = find_best_strategy(ticker_name='EXS1.DE', start='2014-11-05', end='2015-03-31', ngen=5)

Running genetic algorithm ...


C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	min     	max    	mean    	std    
0  	1000  	-115.186	25.2634	-23.1293	50.9981
1  	583   	-110.166	25.2634	5.76885 	30.2291
2  	589   	-110.392	27.0993	13.545  	24.1657
3  	596   	-113.937	27.2517	17.231  	21.0143
4  	600   	-109.918	27.2517	19.7877 	15.5901
5  	624   	-110.699	27.418 	18.167  	22.9382
Net profit of buy and hold: 23.655000000000005
Running backtesting ...
Best individual: [0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]
Best average net profit: 18.097198925006055
Wall time: 7min 49s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='EXS1.DE', start='2015-04-01', end='2015-07-30')
results['EXS1.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='EXS1.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='EXS1.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='EXS1.DE', start='2015-09-28', end='2016-01-26')
results['EXS1.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='EXS1.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='EXS1.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='EXS1.DE', start='2016-03-26', end='2016-07-24')
results['EXS1.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='EXS1.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='EXS1.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='EXS1.DE', start='2016-09-22', end='2017-01-20')
results['EXS1.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='EXS1.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='EXS1.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='EXS1.DE', start='2017-03-21', end='2017-07-19')
results['EXS1.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='EXS1.DE', start='2017-03-21', end='2017-07-19')

## Adidas

In [7]:
%%time
best_individual = find_best_strategy(ticker_name='ADS.DE', start='2014-11-05', end='2015-03-31', ngen=5)

C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Running genetic algorithm ...
gen	nevals	min     	max    	mean   	std    
0  	1000  	-64.7321	21.1204	7.93255	9.56094
1  	583   	-59.8479	21.1204	11.9679	6.42473
2  	589   	-65.0411	21.1204	13.9905	5.65471
3  	596   	-59.9513	21.5717	15.0601	5.25186
4  	600   	-48.5503	21.5717	16.2693	4.62869
5  	624   	-63.043 	21.5717	17.2976	5.08048
Net profit of buy and hold: 15.016725000000003
Running backtesting ...
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]
Best average net profit: 7.6983744363052855
Wall time: 7min 52s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ADS.DE', start='2015-04-01', end='2015-07-30')
results['ADS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ADS.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ADS.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ADS.DE', start='2015-09-28', end='2016-01-26')
results['ADS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ADS.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ADS.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ADS.DE', start='2016-03-26', end='2016-07-24')
results['ADS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ADS.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ADS.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ADS.DE', start='2016-09-22', end='2017-01-20')
results['ADS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ADS.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ADS.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ADS.DE', start='2017-03-21', end='2017-07-19')
results['ADS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ADS.DE', start='2017-03-21', end='2017-07-19')

## Allianz

In [8]:
%%time
best_individual = find_best_strategy(ticker_name='ALV.DE', start='2014-11-05', end='2015-03-31', ngen=5)

C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Running genetic algorithm ...
gen	nevals	min    	max    	mean    	std    
0  	1000  	-125.65	23.8824	-9.68521	46.0219
1  	583   	-120.758	23.8824	10.7798 	21.7268
2  	589   	-120.593	24.9006	16.2579 	16.5536
3  	596   	-118.337	24.9006	18.3021 	15.4589
4  	600   	-117.84 	24.9287	20.1534 	9.72352
5  	624   	-123.191	24.9006	20.0142 	13.1844
Net profit of buy and hold: 24.875375000000002
Running backtesting ...
Best individual: [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0]
Best average net profit: 18.47640204584431
Wall time: 8min 15s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ALV.DE', start='2015-04-01', end='2015-07-30')
results['ALV.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ALV.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ALV.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ALV.DE', start='2015-09-28', end='2016-01-26')
results['ALV.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ALV.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ALV.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ALV.DE', start='2016-03-26', end='2016-07-24')
results['ALV.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ALV.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ALV.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ALV.DE', start='2016-09-22', end='2017-01-20')
results['ALV.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ALV.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ALV.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ALV.DE', start='2017-03-21', end='2017-07-19')
results['ALV.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ALV.DE', start='2017-03-21', end='2017-07-19')

## BASF

In [9]:
%%time
best_individual = find_best_strategy(ticker_name='BAS.DE', start='2014-11-05', end='2015-03-31', ngen=5)

C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Running genetic algorithm ...
gen	nevals	min     	max    	mean   	std    
0  	1000  	-77.0245	19.3883	1.04307	21.7866
1  	583   	-71.5517	22.5203	10.8129	9.7314 
2  	589   	-63.878 	22.5203	13.7616	7.85241
3  	596   	-70.906 	21.9335	14.3675	9.92067
4  	600   	-68.4068	21.9335	14.961 	10.145 
5  	624   	-73.0566	21.9335	13.218 	16.0407
Net profit of buy and hold: 17.373325
Running backtesting ...
Best individual: [1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0]
Best average net profit: 8.428810952608835
Wall time: 6min 51s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='BAS.DE', start='2015-04-01', end='2015-07-30')
results['BAS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='BAS.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='BAS.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='BAS.DE', start='2015-09-28', end='2016-01-26')
results['BAS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='BAS.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='BAS.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='BAS.DE', start='2016-03-26', end='2016-07-24')
results['BAS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='BAS.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='BAS.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='BAS.DE', start='2016-09-22', end='2017-01-20')
results['BAS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='BAS.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='BAS.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='BAS.DE', start='2017-03-21', end='2017-07-19')
results['BAS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='BAS.DE', start='2017-03-21', end='2017-07-19')

## Deutsche Bank

In [10]:
%%time
best_individual = find_best_strategy(ticker_name='DBK.DE', start='2014-11-05', end='2015-03-31', ngen=5)

C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Running genetic algorithm ...
gen	nevals	min     	max    	mean    	std    
0  	1000  	-31.4787	7.38938	-4.11838	12.7531
1  	583   	-29.2112	7.16883	2.95294 	6.33082
2  	589   	-30.1919	7.30423	4.80228 	4.68458
3  	596   	-29.1048	7.54977	5.57224 	4.24519
4  	600   	-30.1919	7.54977	5.87581 	4.44416
5  	624   	-29.7908	7.54977	5.80664 	5.25233
Net profit of buy and hold: 6.260949999999999
Running backtesting ...
Best individual: [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1]
Best average net profit: 5.111292371973835
Wall time: 8min 54s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DBK.DE', start='2015-04-01', end='2015-07-30')
results['DBK.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DBK.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DBK.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DBK.DE', start='2015-09-28', end='2016-01-26')
results['DBK.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DBK.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DBK.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DBK.DE', start='2016-03-26', end='2016-07-24')
results['DBK.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DBK.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DBK.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DBK.DE', start='2016-09-22', end='2017-01-20')
results['DBK.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DBK.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DBK.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DBK.DE', start='2017-03-21', end='2017-07-19')
results['DBK.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DBK.DE', start='2017-03-21', end='2017-07-19')

## Deutsche Telekom

In [11]:
%%time
best_individual = find_best_strategy(ticker_name='DTE.DE', start='2014-11-05', end='2015-03-31', ngen=5)

C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Running genetic algorithm ...
gen	nevals	min   	max    	mean    	std    
0  	1000  	-12.76	4.13692	-1.52999	5.57376
1  	583   	-12.2756	4.13692	1.7754  	2.83098
2  	589   	-12.0579	4.13692	2.58618 	2.39765
3  	596   	-11.0441	4.13692	3.17297 	1.72363
4  	600   	-11.7157	4.13692	3.34095 	1.71612
5  	624   	-11.6869	4.13692	3.51683 	1.6528 
Net profit of buy and hold: 3.53025
Running backtesting ...
Best individual: [1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0]
Best average net profit: 2.4470977477703313
Wall time: 8min 20s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DTE.DE', start='2015-04-01', end='2015-07-30')
results['DTE.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DTE.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DTE.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DTE.DE', start='2015-09-28', end='2016-01-26')
results['DTE.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DTE.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DTE.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DTE.DE', start='2016-03-26', end='2016-07-24')
results['DTE.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DTE.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DTE.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DTE.DE', start='2016-09-22', end='2017-01-20')
results['DTE.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DTE.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DTE.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DTE.DE', start='2017-03-21', end='2017-07-19')
results['DTE.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DTE.DE', start='2017-03-21', end='2017-07-19')

## SAP

In [12]:
%%time
best_individual = find_best_strategy(ticker_name='SAP.DE', start='2014-11-05', end='2015-03-31', ngen=5)

C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Running genetic algorithm ...
gen	nevals	min     	max    	mean    	std    
0  	1000  	-66.4764	11.7965	-7.05599	23.5441
1  	583   	-66.245 	11.9351	4.33623 	9.48753
2  	589   	-64.0441	11.9351	7.33205 	8.64112
3  	596   	-65.8959	12.3746	8.90186 	9.1388 
4  	600   	-64.3921	12.6269	10.1129 	6.10793
5  	624   	-64.6708	12.6269	9.84516 	9.28874
Net profit of buy and hold: 11.86665
Running backtesting ...
Best individual: [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0]
Best average net profit: 9.789801019766188
Wall time: 8min 27s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='SAP.DE', start='2015-04-01', end='2015-07-30')
results['SAP.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='SAP.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='SAP.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='SAP.DE', start='2015-09-28', end='2016-01-26')
results['SAP.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='SAP.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='SAP.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='SAP.DE', start='2016-03-26', end='2016-07-24')
results['SAP.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='SAP.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='SAP.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='SAP.DE', start='2016-09-22', end='2017-01-20')
results['SAP.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='SAP.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='SAP.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='SAP.DE', start='2017-03-21', end='2017-07-19')
results['SAP.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='SAP.DE', start='2017-03-21', end='2017-07-19')

## Dow Jones Index

In [5]:
%%time
best_individual = find_best_strategy(ticker_name='DJI', start='2014-11-05', end='2015-03-31', ngen=5)

Running genetic algorithm ...
gen	nevals	min     	max    	mean    	std   
0  	1000  	-21296.1	1176.94	-11214.5	9395.8
1  	583   	-20774.9	1268.14	-4647.74	7970.39
2  	589   	-20516  	1268.14	-1254.47	4950.01
3  	596   	-20701.4	1268.14	-832.34 	4681.59
4  	600   	-20435.3	1335.93	-746.339	4809.54
5  	624   	-20281  	1416.73	-479.207	4579.01
Net profit of buy and hold: 504.09975
Running backtesting ...
Best individual: [1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0]
Best average net profit: -422.62888881572076
Wall time: 10min 21s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DJI', start='2015-04-01', end='2015-07-30')
results['DJI'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DJI', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DJI', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DJI', start='2015-09-28', end='2016-01-26')
results['DJI'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DJI', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DJI', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DJI', start='2016-03-26', end='2016-07-24')
results['DJI'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DJI', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DJI', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DJI', start='2016-09-22', end='2017-01-20')
results['DJI'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DJI', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DJI', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DJI', start='2017-03-21', end='2017-07-19')
results['DJI'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DJI', start='2017-03-21', end='2017-07-19')

## Cisco

In [13]:
%%time
best_individual = find_best_strategy(ticker_name='CSCO', start='2014-11-05', end='2015-03-31', ngen=5)

C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Running genetic algorithm ...
gen	nevals	min     	max    	mean    	std    
0  	1000  	-27.2333	3.42215	-3.37432	7.55707
1  	583   	-27.3809	3.87963	-0.105589	3.09822
2  	589   	-27.0575	4.29708	0.88527  	2.98244
3  	596   	-25.9853	4.29708	1.68359  	2.22495
4  	600   	-25.6   	4.6056 	2.07771  	2.60936
5  	624   	-26.7515	4.57898	2.59989  	2.3379 
Net profit of buy and hold: 2.3601250000000022
Running backtesting ...
Best individual: [1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0]
Best average net profit: 0.17176941144842778
Wall time: 8min 1s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='CSCO', start='2015-04-01', end='2015-07-30')
results['CSCO'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='CSCO', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='CSCO', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='CSCO', start='2015-09-28', end='2016-01-26')
results['CSCO'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='CSCO', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='CSCO', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='CSCO', start='2016-03-26', end='2016-07-24')
results['CSCO'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='CSCO', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='CSCO', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='CSCO', start='2016-09-22', end='2017-01-20')
results['CSCO'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='CSCO', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='CSCO', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='CSCO', start='2017-03-21', end='2017-07-19')
results['CSCO'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='CSCO', start='2017-03-21', end='2017-07-19')

## DowDuPont

In [14]:
%%time
best_individual = find_best_strategy(ticker_name='DD', start='2014-11-05', end='2015-03-31', ngen=5)

C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Running genetic algorithm ...
gen	nevals	min     	max    	mean    	std    
0  	1000  	-66.6761	8.79597	-3.02145	12.2461
1  	583   	-57.4131	8.52337	1.10563 	6.21409
2  	589   	-57.852 	8.70962	2.59996 	6.3312 
3  	596   	-58.1327	9.97855	4.13264 	5.32181
4  	600   	-56.4622	10.5383	5.39373 	5.122  
5  	624   	-56.5588	11.2757	6.5043  	3.87982
Net profit of buy and hold: 1.984049999999998
Running backtesting ...
Best individual: [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1]
Best average net profit: -4.903473345476557
Wall time: 7min 36s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DD', start='2015-04-01', end='2015-07-30')
results['DD'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DD', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DD', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DD', start='2015-09-28', end='2016-01-26')
results['DD'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DD', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DD', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DD', start='2016-03-26', end='2016-07-24')
results['DD'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DD', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DD', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DD', start='2016-09-22', end='2017-01-20')
results['DD'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DD', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DD', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DD', start='2017-03-21', end='2017-07-19')
results['DD'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DD', start='2017-03-21', end='2017-07-19')

## JPMorgan Chase

In [15]:
%%time
best_individual = find_best_strategy(ticker_name='JPM', start='2014-11-05', end='2015-03-31', ngen=5)

C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Running genetic algorithm ...
gen	nevals	min     	max    	mean    	std    
0  	1000  	-64.8561	4.85235	-29.5092	27.3119
1  	583   	-58.9731	4.85235	-10.392 	21.1482
2  	589   	-59.5864	6.61972	-2.50853	13.3604
3  	596   	-56.7556	6.61972	-0.170918	11.0186
4  	600   	-58.5323	6.61972	1.18592  	10.4578
5  	624   	-57.1273	7.15035	1.76917  	11.3068
Net profit of buy and hold: 0.6819499999999977
Running backtesting ...
Best individual: [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0]
Best average net profit: 0.20663458037902832
Wall time: 6min 47s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='JPM', start='2015-04-01', end='2015-07-30')
results['JPM'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='JPM', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='JPM', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='JPM', start='2015-09-28', end='2016-01-26')
results['JPM'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='JPM', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='JPM', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='JPM', start='2016-03-26', end='2016-07-24')
results['JPM'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='JPM', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='JPM', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='JPM', start='2016-09-22', end='2017-01-20')
results['JPM'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='JPM', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='JPM', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='JPM', start='2017-03-21', end='2017-07-19')
results['JPM'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='JPM', start='2017-03-21', end='2017-07-19')

## Microsoft

In [16]:
%%time
best_individual = find_best_strategy(ticker_name='MSFT', start='2014-11-05', end='2015-03-31', ngen=5)

C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Running genetic algorithm ...
gen	nevals	min     	max    	mean    	std    
0  	1000  	-42.8191	4.14907	-5.35359	4.42281
1  	583   	-41.0078	4.8484 	-2.81925	3.28415
2  	589   	-38.9878	5.0219 	-1.09405	3.17311
3  	596   	-38.9137	6.40558	0.0181348	3.32848
4  	600   	-37.2673	6.40558	1.07238  	2.91617
5  	624   	-38.4181	6.4949 	2.1141   	2.99154
Net profit of buy and hold: -5.557099999999999
Running backtesting ...
Best individual: [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
Best average net profit: -1.1130238119740947
Wall time: 8min 24s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='MSFT', start='2015-04-01', end='2015-07-30')
results['MSFT'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='MSFT', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='MSFT', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='MSFT', start='2015-09-28', end='2016-01-26')
results['MSFT'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='MSFT', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='MSFT', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='MSFT', start='2016-03-26', end='2016-07-24')
results['MSFT'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='MSFT', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='MSFT', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='MSFT', start='2016-09-22', end='2017-01-20')
results['MSFT'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='MSFT', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='MSFT', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='MSFT', start='2017-03-21', end='2017-07-19')
results['MSFT'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='MSFT', start='2017-03-21', end='2017-07-19')

## Nike

In [17]:
%%time
best_individual = find_best_strategy(ticker_name='NKE', start='2014-11-05', end='2015-03-31', ngen=5)

C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Running genetic algorithm ...
gen	nevals	min     	max   	mean    	std    
0  	1000  	-54.1323	5.1097	-5.67382	14.9025
1  	583   	-53.6134	6.98415	0.252615	5.64543
2  	589   	-53.6134	6.98415	1.46728 	5.25846
3  	596   	-54.9266	5.43745	1.68592 	6.19162
4  	600   	-53.5904	6.46683	2.11797 	5.69406
5  	624   	-53.2068	7.1762 	2.21497 	6.23271
Net profit of buy and hold: 3.504725000000004
Running backtesting ...
Best individual: [1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
Best average net profit: 0.5724479945495122
Wall time: 7min 47s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='NKE', start='2015-04-01', end='2015-07-30')
results['NKE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='NKE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='NKE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='NKE', start='2015-09-28', end='2016-01-26')
results['NKE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='NKE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='NKE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='NKE', start='2016-03-26', end='2016-07-24')
results['NKE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='NKE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='NKE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='NKE', start='2016-09-22', end='2017-01-20')
results['NKE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='NKE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='NKE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='NKE', start='2017-03-21', end='2017-07-19')
results['NKE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='NKE', start='2017-03-21', end='2017-07-19')

## UnitedHealth

In [18]:
%%time
best_individual = find_best_strategy(ticker_name='UNH', start='2014-11-05', end='2015-03-31', ngen=5)

C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Jonathan\Anaconda3\lib\site-packages\deap-1.3.1-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Running genetic algorithm ...
gen	nevals	min     	max    	mean    	std    
0  	1000  	-119.661	29.1084	-6.48983	41.4597
1  	583   	-119.831	29.1084	10.9038 	23.4614
2  	589   	-122.823	31.589 	16.9294 	20.0234
3  	596   	-113.997	31.589 	19.5047 	18.2909
4  	600   	-119.803	31.589 	21.5079 	17.1257
5  	624   	-120.051	32.5916	23.8932 	12.2873
Net profit of buy and hold: 23.957075000000003
Running backtesting ...
Best individual: [1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1]
Best average net profit: 17.049636161361594
Wall time: 7min 52s


In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='UNH', start='2015-04-01', end='2015-07-30')
results['UNH'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='UNH', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='UNH', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='UNH', start='2015-09-28', end='2016-01-26')
results['UNH'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='UNH', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='UNH', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='UNH', start='2016-03-26', end='2016-07-24')
results['UNH'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='UNH', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='UNH', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='UNH', start='2016-09-22', end='2017-01-20')
results['UNH'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='UNH', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='UNH', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='UNH', start='2017-03-21', end='2017-07-19')
results['UNH'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='UNH', start='2017-03-21', end='2017-07-19')